**AUGMENTATION SCRIPT ON COLAB**

1. Upload the augmentation script into your Google Drive   
2. Mount your drive & navigate to your augmentation script folder
3. Replace your Roboflow dataset API (if needed)
4. Configure the augmentation modules as per your need
5. Adjust the augmentation parameters accordingly

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Navigate to script location
%cd "script_dir"

In [ ]:
# Install libraries that do not available in Colab
!pip install roboflow

In [ ]:
# Configure the augmentation modules as per your need
# Mainly augmentation done on train dataset
def get_train_transforms():
    transform = A.ReplayCompose(
        [
            # A.Resize(1024, 1024, p=1.0),
            A.LongestMaxSize(max_size=1024, interpolation=cv2.INTER_LINEAR, p=1.0),
            ########################################## random scale implementation of object-of-interests ##################################################
            # Additional: Random scale and padded to adapt various scales of object of interest
            A.RandomScale(scale_limit=(-0.5, -0.8),interpolation=cv2.INTER_LINEAR,mask_interpolation=cv2.INTER_NEAREST,area_for_downscale="image", p=0.4),
            # A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=cv2.BORDER_CONSTANT, fill=0, fill_mask=0, p=1.0),
            ################################################################################################################################################
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.3),
            A.Rotate(limit=15, border_mode=cv2.BORDER_CONSTANT, p=0.4),
            A.ToGray(p=0.2),
            A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=25, p=0.3),
            A.RandomBrightnessContrast(p=0.4),
            A.MotionBlur(blur_limit=(3, 5), p=0.3),
            A.Emboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=0.5),
            A.CLAHE(clip_limit=(1, 4), tile_grid_size=(8, 8), p=0.8),
            A.ElasticTransform(alpha=100, sigma=50, p=0.5),
            A.Downscale(scale_range=(0.5, 0.75), interpolation_pair={'downscale': cv2.INTER_NEAREST, 'upscale': cv2.INTER_LINEAR}, p=0.5),
        ],
        bbox_params=A.BboxParams(
            format="coco",
            label_fields=["category_ids"]
        )
    )
    return transform

In [ ]:
# No augmentation for valid and test dataset (just standard resize)
def get_valid_transforms():
    transform = A.ReplayCompose(
        [
            # A.Resize(1024, 1024, p=1.0),
            A.LongestMaxSize(max_size=1024, interpolation=cv2.INTER_LINEAR, p=1.0),
        ],
        bbox_params=A.BboxParams(
            format="coco",
            label_fields=["category_ids"]
        )
    )
    return transform

In [ ]:
# Import on-premise libraries
import os
import cv2
import augmenting_coco_1 as ag            # your augmentation script name
import albumentations as A

# Replace your Roboflow dataset API
def roboflow_dataset():
  from roboflow import Roboflow
  rf = Roboflow(api_key="")
  project = rf.workspace("").project("")
  version = project.version(8)
  dataset = version.download("coco-segmentation")


  return dataset

In [ ]:
PROJECT_NAME = 'Project'

# Adjust your augmentation parameters and run the script
ag.main(
    data_type       = 'roboflow',                                                               # 'local' (local drive) and 'roboflow'
    local_data      = None,
    roboflow_data   = roboflow_dataset().location,                                              # roboflow_dataset().location or None
    result_path     = os.path.join('path', PROJECT_NAME),
    data_dist       = {'train': 3, 'valid': 1, 'test': 1},
    train_transform = get_train_transforms(),
    valid_transform = get_valid_transforms(),
)